In [1]:
import numpy as np
import time

In [2]:
from numpy import linalg as la

In [3]:
np.set_printoptions(precision=2)

In [4]:
A = np.random.rand(2000,2000)

In [5]:
B = np.random.rand(2000,2000)

In [6]:
def strass(A, B, steps):
    
    #Check Dimensions
    (m,n) = A.shape
    (nn, p) = B.shape
    if n != nn: raise ValueError("incompatible dimensions")
    C = np.zeros((m,p))
    
    #Base case
    if steps == 0 or m ==1 or n ==1 or p == 1:
        C = np.dot(A,B)
        return C
    
    #Dynamic peeling
    if m % 2 == 1:
        C[:m-1, :] = strass(A[:m-1,:],B, steps)
        C[m-1,:] = A[m-1,:]@B
        return C
    if n % 2 == 1:
        C = strass(A[:, :n-1], B[:n-1,:], steps)
        C = C + np.outer(A[:,n-1],B[n-1,:])
        return C
    if p % 2 == 1:
        C[:, :p-1] = strass(A, B[:,:p-1], steps)
        C[:,p-1] = A@B[:,p-1]
        return C
    
    # divide when m, n and p are all even
    m2 = int(m/2)
    n2 = int(n/2)
    p2 = int(p/2)
    A11 = A[:m2,:n2] 
    A12 = A[:m2,n2:]
    A21 = A[m2:,:n2] 
    A22 = A[m2:,n2:]
    B11 = B[:n2,:p2]   
    B12 = B[:n2,p2:]
    B21 = B[n2:,:p2] 
    B22 = B[n2:,p2:]
    
    # conquer
    M1 = strass(A11, B12-B22   ,steps-1)
    M2 = strass(A11+A12, B22   ,steps-1)
    M3 = strass(A21+A22,B11    ,steps-1)
    M4 = strass(A22    ,B21-B11,steps-1)
    M5 = strass(A11+A22,B11+B22,steps-1)
    M6 = strass(A12-A22,B21+B22,steps-1)
    M7 = strass(A11-A21,B11+B12,steps-1)
    
    # conquer    
    C[:m2,:p2] = M5+M4-M2+M6 
    C[:m2,p2:] = M1+M2 
    C[m2:,:p2] = M3+M4
    C[m2:,p2:] = M1+M5-M3-M7
    
    return C

In [8]:
start = time.time()
C = strass(A,B, 2)
end = time.time()
print(end - start)

0.24393081665039062


In [8]:
start_norm = time.time()
Cnorm = A*B
end_norm = time.time()
print(end_norm - start_norm)

0.022013425827026367


In the cell below we can see that the accuracy of the algorithm for the given matrix is around machine epsilon

In [9]:
la.norm(C-A@B, 'fro')/la.norm(C)

7.131791135719216e-16

In [10]:
# Accuracy and Stability of Numerical Algorithms second edition